In [1]:
# We start by loading the libraries we're going to use
import folium, json
import pandas as pd

In [2]:
# Here we store the paths to the data files
# Remember to change this in case your folder structure is different
geo_json_path = "../fyp2022p0105/data/raw/shapefiles/dk.geojson"
corona_df = pd.read_csv("../fyp2022p0105/data/raw/corona/dk_corona.csv", sep = "\t")

# Loading the country metadata directly, since it is a JSON file
with open("../fyp2022p0105/data/raw/metadata/dk_metadata.json", 'r') as f:
   country_metadata = json.load(f)

In [3]:
# Time to look at some maps
# First, let's initialize folium by finding the geographic center of the country and an appropriate zoom level
m = folium.Map(location = [56, 10], zoom_start = 7, crs = "EPSG3857")

m

In [4]:
# We now load the geoJSON which contains the bonduaries for the country's regions
folium.GeoJson(geo_json_path, name = "geojson").add_to(m)

folium.LayerControl().add_to(m)

m

In [5]:
# Next task, we need to convert the region names in the corona data with the iso3166-2 codes
# To do so, first we need a dictionary mapping the region name to the code
region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}

# Then, we use the Series.map function
corona_df["region"] = corona_df["region_code"].map(region_map)

# Next task, we can now calculate how many cases there were in total for the region
corona_df_by_region = corona_df.groupby(by = "region")["hospitalized_addition"].sum().reset_index()

# If we want to add population data, we need to create another dictionary from the country metadata
# In this case, we need to map from the region iso3166-2 code to the region's population
population_map = {country_metadata["country_metadata"][i]["iso3166-2_code"]: country_metadata["country_metadata"][i]["population"] for i in range(len(country_metadata["country_metadata"]))}

# Then we can map, just like before
corona_df_by_region["population"] = corona_df_by_region["region"].map(population_map)

corona_df_by_region

,region,hospitalized_addition,population
0,DK-81,767,589936
1,DK-82,1788,1326340
2,DK-83,1312,1223105
3,DK-84,6041,1846023
4,DK-85,2104,837359


In [6]:
# Next task, mapping the number of cases in a choropeth map
# We start by initializing the folium map
m_cases = folium.Map(location = [56, 10], zoom_start = 7)

# Then we use the Choropleth function, which requires: the path to the geoJSON (the shapes),
# a name for the layer, the dataframe containing the data, the columns we want to focus on,
# the attribute from the geoJSON which we use to map the data with the corresponding geometry
# in our case this is the iso code. Finally, we specify the color map, and the opacity (so that
# we can still see the map beneath), and a title.
folium.Choropleth(
    geo_data = geo_json_path,
    name = "cases",
    data = corona_df_by_region,
    columns = ["region", "hospitalized_addition"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Hospitalized",
).add_to(m_cases)

m_cases

In [7]:
# Next task: is more cases bad, or could it be explained simpyl by a larger population?
# Let's make another map using the exact same function, this time we plot population
# instead of cases. We also use a different color map.
m_pop = folium.Map(location = [56, 10], zoom_start = 7)

folium.Choropleth(
    geo_data = geo_json_path,
    name = "population",
    data = corona_df_by_region,
    columns = ["region", "population"],
    key_on = "properties.iso_3166_2",
    fill_color = "YlGn",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Population",
).add_to(m_pop)

m_pop

In [8]:
# Final task, we create a new column in our data frame with the cases per capita
corona_df_by_region["hospitalized_pc"] = corona_df_by_region["hospitalized_addition"] / corona_df_by_region["population"]

# And we plot it just like we did before
m_cases_pc = folium.Map(location = [56, 10], zoom_start = 7)

folium.Choropleth(
    geo_data = geo_json_path,
    name = "cases",
    data = corona_df_by_region,
    columns = ["region", "hospitalized_pc"],
    key_on = "properties.iso_3166_2",
    fill_color = "OrRd",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Hospitalized PC",
).add_to(m_cases_pc)

m_cases_pc